In [3]:
import json
import os
import math
import librosa
from tensorflow.keras.models import load_model
import numpy as np
from pydub import AudioSegment

import soundfile as sf
from pedalboard.io import AudioFile
from pedalboard import *
import noisereduce as nr

### Code

In [23]:
# model = load_model("./model/custom_cnn_0.0031.h5")
# SAMPLE_RATE = 22050
model = load_model("./model/cnn__genre_detection_41100hz_0.93.h5")
SAMPLE_RATE = 41100
TRACK_DURATION = 30  # second
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
file_path = "../mfcc_genre_detection/datasets/jazz/jazz.00033.wav"
# file_path = "../audio_tester/armada - hargai aku-[AudioTrimmer.com].mp3"
# file_path = "../../../Musics/2022+/SZA - Kill Bill (Audio).mp3"
# file_path = "../../../Musics/gama - satu ataw dua.mp3"
# file_path = "../../../Musics/2020s/Ava Max - Salt (Lyrics).mp3"
# file_path = "../../../Musics/2020s/Qorygore - Sampurasun (Lyrics) ft. Pretty Rico.mp3"
# file_path = "../../../Musics/2020s/Dj Quads - Punch.mp3"
# file_path = "../../../Musics/2020s/mxmtoon - ok on your own (ft. Carly Rae Jepsen)  audio .mp3"
# file_path = "audio1_enhenced.wav"
# file_path = "soundfile.wav"

def extract_mfcc(file_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    # dictionary to store mapping, labels, and MFCCs

    data = []
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    # signal = signal[(SAMPLE_RATE * 40):(SAMPLE_RATE * 70)]
    # signal = nr.reduce_noise(y=signal, sr=SAMPLE_RATE)
    

    for d in range(num_segments):
        # calculate start and finish sample for current segment
        start = samples_per_segment * d
        finish = start + samples_per_segment
        # extract mfcc

        mfcc = librosa.feature.mfcc(

            y=signal[start:finish],
            sr=sample_rate,
            n_mfcc=num_mfcc,
            n_fft=n_fft,

            hop_length=hop_length,
        )

        mfcc = mfcc.T
        # store only mfcc feature with expected number of vectors

        if len(mfcc) == num_mfcc_vectors_per_segment:
            data.append(mfcc.tolist())

    return data

val = extract_mfcc(file_path=file_path, num_segments=10)

# print(val)

val = np.array(val)

print(val.shape)

val = val[..., np.newaxis]


percentage = 0

genres = {
    "Blues": 0,
    "Classical": 0,
    "Country": 0,
    "Disco": 0,
    "HipHop": 0,
    "Jazz": 0,
    "Metal": 0,
    "Pop": 0,
    "Reggae": 0,
    "Rock": 0,
}

genres_list = list(genres.keys())

accuracy = 0

sum_accuracy = 0

min_acc = 0
# for i in range(val.shape[0]) :
#     prediction = model.predict(segmen)
#     print(np.max(prediction))
#     accuracy = accuracy + np.max(prediction)

# print(accuracy)


prediction = model.predict(val)
# print(prediction.shape)

for i in range(prediction.shape[0]):
    #     genres[current_genre] = value

    for key, v in enumerate(prediction[i]):

        # genres[genres_list[key]] = genres[genres_list[key]] + v
        genres[genres_list[key]] = ((genres[genres_list[key]] * 3)+ v) / 4
        # if key == 5:
        #     # print(prediction[i][key])
        #     sum_accuracy = sum_accuracy + prediction[i][key]

    # print('max value ==>', np.max(prediction[i]))
    # print('genre ==>', np.argmax(prediction[i]))
    # print('min value ==>', np.min(prediction[i]))
    # print('genre ==>', np.argmin(prediction[i]))
    # min_acc = min_acc + np.min(prediction[i])
    # accuracy = accuracy + np.max(prediction[i])
# print('accuracy ==>', accuracy)
# print('min accuracy  ==>', min_acc)
# print('sum_accuracy ==>', sum_accuracy)

print("<<========= Genre Detection Accuracy =======>>")
# print(genres)
total_val = 0
for genre in genres:
    print("{}\t\t==> {}%".format(genre, round(genres[genre] * 10, 2)))
    total_val = total_val + genres[genre]

print("total_val ==>", total_val)

# print(
#     "mfcc_val ==> {} \nprediction ==> {}\npredicted_index ==> {}".format(
#         val.shape, prediction, predicted_index
#     )
# )
# for v in prediction:
#     print(v)
#     print('==='*30)
#     print(np.max(v))
#     print('==='*30)

(10, 241, 13)
1/1 [==============================] - 0s 362ms/step
<<========= Genre Detection Accuracy =======>>
Blues		==> 0.0%
Classical		==> 0.0%
Country		==> 0.0%
Disco		==> 0.0%
HipHop		==> 0.0%
Jazz		==> 9.44%
Metal		==> 0.0%
Pop		==> 0.0%
Reggae		==> 0.0%
Rock		==> 0.0%
total_val ==> 0.943686477461976


### Test Code

In [ ]:
nilai = np.arange(0,10)
genres = {"Blues" : 0,
            "Classical" : 0,
            "Country" : 0,
            "Disco" : 0,
            "HipHop" : 0,
            "Jazz" : 0,"Metal" : 0,
            "Pop" : 0,
            "Reggae" : 0,
            "Rock" : 0}

# genre_index = 0
# for i in range(nilai.size):
#     current_genre = list(genres.keys())[genre_index]
#     genres[current_genre] = i
#     genre_index += 1

for key, val in enumerate(genres):
  print(key) 
  print(val) 

pri

In [39]:
signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
signal = signal[(SAMPLE_RATE * 40):(SAMPLE_RATE * 70)]
sf.write("test.wav", signal, samplerate=sample_rate)

In [53]:
sr=44100
with AudioFile('scipy.wav').resampled_to(sr) as f:
    audio = f.read(f.frames)

reduced_noise = nr.reduce_noise(y=audio, sr=sr, stationary=True, prop_decrease=0.5)

board = Pedalboard([
    NoiseGate(threshold_db=-30, ratio=1.5, release_ms=250),
    Compressor(threshold_db=-15, ratio=2.5),
    LowShelfFilter(cutoff_frequency_hz=400, gain_db=10, q=1),
    Gain(gain_db=10)
])

effected = board(reduced_noise, sr)


with AudioFile('audio1_enhenced.wav', 'w', sr, effected.shape[0]) as f:
    f.write(effected)